SKRIPSI

# Import library

In [1]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
import re
import nltk

from nltk import ngrams
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, validation_curve,RandomizedSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from matplotlib_venn import venn2
from time import time

np.random.seed(0)
stopwords = nltk.corpus.stopwords.words('english')
poster_Stemmer = nltk.PorterStemmer()


# Baca file csv

In [2]:
#baca csv
data_train_amazon = pd.read_csv('Amazon_Train.csv')
data_train_yelp = pd.read_csv('Yelp_Train.csv')
data_test_amazon = pd.read_csv('Amazon_Test.csv')
data_test_yelp = pd.read_csv('Yelp_Test.csv')

In [3]:
print(data_train_amazon.head())
print("\n")
print(data_train_yelp.head())

   Label                                           Sentimen
0      0  Buyer beware This is a self-published book, an...
1      0  The Worst! A complete waste of time. Typograph...
2      0  Oh please I guess you have to be a romance nov...
3      0  Awful beyond belief! I feel I have to write to...
4      0  Another Abysmal Digital Copy Rather than scrat...


   Label                                           Sentimen
0      0  I don't know what Dr. Goldberg was like before...
1      0  I'm writing this review to give you a heads up...
2      0  Owning a driving range inside the city limits ...
3      0  This place is absolute garbage...  Half of the...
4      0  Used to go there for tires, brakes, etc.  Thei...


In [4]:
print(data_test_amazon.head())
print("\n")
print(data_test_yelp.head())

   Label                                           Sentimen
0      0  Overly complicated Being both a U.S. history b...
1      0  Terrible Disappointment -- parts don't fit My ...
2      0  Didn't hold up.....very disappointed I bought ...
3      0  gene hates jezebel i love JLJ but this compila...
4      0  Nice toy but ... My six-year-old loves space a...


   Label                                           Sentimen
0      0  My wife and I used to love Arriba's, til recen...
1      0  You get what you pay for.  The food is inexpen...
2      0  Unfortunately, yesterday's visit was one of th...
3      0  I went into the Scottsdale location yesterday....
4      0  It takes a lot for me to write a review and bl...


# Preprocessing 

In [5]:
data_Preprocessing_Amazon_Train = data_train_amazon.Sentimen.astype(str)
data_Preprocessing_Amazon_Train = data_Preprocessing_Amazon_Train.apply(lambda x: x.lower())
data_Preprocessing_Amazon_Train = data_Preprocessing_Amazon_Train.apply(lambda x: re.sub(r"\d", "", x))   
data_Preprocessing_Amazon_Train = data_Preprocessing_Amazon_Train.apply(lambda x: x.translate(str.maketrans('','',string.punctuation))) #punctuation  
data_Preprocessing_Amazon_Train = data_Preprocessing_Amazon_Train.apply(lambda x: " ".join(re.findall("[a-zA-Z]+", x)))

data_Preprocessing_Yelp_Train = data_train_yelp.Sentimen.astype(str)
data_Preprocessing_Yelp_Train = data_Preprocessing_Yelp_Train.apply(lambda x: x.lower())
data_Preprocessing_Yelp_Train = data_Preprocessing_Yelp_Train.apply(lambda x: re.sub(r"\d", "", x))   
data_Preprocessing_Yelp_Train = data_Preprocessing_Yelp_Train.apply(lambda x: x.translate(str.maketrans('','',string.punctuation))) #punctuation  
data_Preprocessing_Yelp_Train = data_Preprocessing_Yelp_Train.apply(lambda x: " ".join(re.findall("[a-zA-Z]+", x)))

In [6]:
data_Preprocessing_Amazon_Train.head()

0    buyer beware this is a selfpublished book and ...
1    the worst a complete waste of time typographic...
2    oh please i guess you have to be a romance nov...
3    awful beyond belief i feel i have to write to ...
4    another abysmal digital copy rather than scrat...
Name: Sentimen, dtype: object

In [7]:
data_Preprocessing_Yelp_Train.head()

0    i dont know what dr goldberg was like before m...
1    im writing this review to give you a heads up ...
2    owning a driving range inside the city limits ...
3    this place is absolute garbage half of the tee...
4    used to go there for tires brakes etc their pr...
Name: Sentimen, dtype: object

In [8]:
data_Preprocessing_Amazon_Test = data_test_amazon.Sentimen.astype(str)
data_Preprocessing_Amazon_Test = data_Preprocessing_Amazon_Test.apply(lambda x: x.lower())
data_Preprocessing_Amazon_Test = data_Preprocessing_Amazon_Test.apply(lambda x: re.sub(r"\d", "", x))   
data_Preprocessing_Amazon_Test = data_Preprocessing_Amazon_Test.apply(lambda x: x.translate(str.maketrans('','',string.punctuation))) #punctuation  
data_Preprocessing_Amazon_Test = data_Preprocessing_Amazon_Test.apply(lambda x: " ".join(re.findall("[a-zA-Z]+", x)))

data_Preprocessing_Yelp_Test = data_test_yelp.Sentimen.astype(str)
data_Preprocessing_Yelp_Test = data_Preprocessing_Yelp_Test.apply(lambda x: x.lower())
data_Preprocessing_Yelp_Test = data_Preprocessing_Yelp_Test.apply(lambda x: re.sub(r"\d", "", x))   
data_Preprocessing_Yelp_Test = data_Preprocessing_Yelp_Test.apply(lambda x: x.translate(str.maketrans('','',string.punctuation))) #punctuation  
data_Preprocessing_Yelp_Test = data_Preprocessing_Yelp_Test.apply(lambda x: " ".join(re.findall("[a-zA-Z]+", x)))

In [9]:
data_Preprocessing_Amazon_Test.head()

0    overly complicated being both a us history buf...
1    terrible disappointment parts dont fit my son ...
2    didnt hold upvery disappointed i bought this o...
3    gene hates jezebel i love jlj but this compila...
4    nice toy but my sixyearold loves space and to ...
Name: Sentimen, dtype: object

In [10]:
data_Preprocessing_Yelp_Test.head()

0    my wife and i used to love arribas til recentl...
1    you get what you pay for the food is inexpensi...
2    unfortunately yesterdays visit was one of the ...
3    i went into the scottsdale location yesterday ...
4    it takes a lot for me to write a review and bl...
Name: Sentimen, dtype: object

In [11]:
Clean_Amazon_Sentiment_Train = data_Preprocessing_Amazon_Train
Clean_Yelp_Sentiment_Train = data_Preprocessing_Yelp_Train
Label_Amazon_Train = data_train_amazon['Label']
Label_Yelp_Train = data_train_yelp['Label']

Clean_Amazon_Sentiment_Test = data_Preprocessing_Amazon_Test
Clean_Yelp_Sentiment_Test = data_Preprocessing_Yelp_Test
Label_Amazon_Test = data_test_amazon['Label']
Label_Yelp_Test = data_test_yelp['Label']

# N-Gram (bigram dan trigram)

In [12]:
def bigram_trigram(Clean_Sentiment) :
    iterator = 0
    Vocabulary = []
    ngram_result = []
    while iterator < len(Clean_Sentiment) :
        ngram = 2
        sentence = Clean_Sentiment[iterator]
        vocab = [sentence[i:i+ngram] for i in range(len(sentence)-ngram+1)]
        temp = vocab

        iterator2 = 0
        while iterator2 < len (vocab):
            if(vocab[iterator2] not in Vocabulary) :
                Vocabulary.append(vocab[iterator2])
            iterator2 = iterator2 + 1

        ngram = 3
        vocab = [sentence[i:i+ngram] for i in range(len(sentence)-ngram+1)]
        temp = temp + vocab
        ngram_result.append(temp)
        iterator2 = 0
        while iterator2 < len (vocab):
            if(vocab[iterator2] not in Vocabulary) :
                Vocabulary.append(vocab[iterator2])
            iterator2 = iterator2 + 1
        iterator = iterator + 1
    
    return ngram_result,Vocabulary


In [13]:
ngram_amazon_train, Vocabulary_Amazon_Train = bigram_trigram(Clean_Amazon_Sentiment_Train)

In [14]:
ngram_yelp_train, Vocabulary_Yelp_Train = bigram_trigram(Clean_Yelp_Sentiment_Train)

In [15]:
ngram_amazon_test, Vocabulary_Amazon_Test = bigram_trigram(Clean_Amazon_Sentiment_Test)

In [16]:
ngram_yelp_test, Vocabulary_Yelp_Test = bigram_trigram(Clean_Yelp_Sentiment_Test)

# Me inisialisasi jumlah frekuensi Gram di Vocabulary

In [17]:
def dictionaryInitialize(Clean_Sentiment, Vocabulary, ngram, jenisdata) :
    wordDictionaryCount = []
    iterator = 0
    while iterator < len(Clean_Sentiment):
        wordDictionaryCount.append(dict.fromkeys(Vocabulary , 0))
        iterator = iterator + 1
    
    iterator = 0
    
    while iterator < len(Clean_Sentiment) :
        for gram in ngram[iterator] :
            if jenisdata == 'test' and gram not in Vocabulary :
                continue
            wordDictionaryCount[iterator][gram] += 1
        iterator = iterator + 1
    
    return wordDictionaryCount

In [18]:
WordDictionaryCount_Amazon_Train = dictionaryInitialize(Clean_Amazon_Sentiment_Train, Vocabulary_Amazon_Train, ngram_amazon_train,'train')

In [19]:
WordDictionaryCount_Yelp_Train = dictionaryInitialize(Clean_Yelp_Sentiment_Train, Vocabulary_Yelp_Train, ngram_yelp_train,'train')

In [20]:
WordDictionaryCount_Amazon_Test = dictionaryInitialize(Clean_Amazon_Sentiment_Test, Vocabulary_Amazon_Train, ngram_amazon_test,'test')

In [21]:
WordDictionaryCount_Yelp_Test = dictionaryInitialize(Clean_Yelp_Sentiment_Test, Vocabulary_Yelp_Train, ngram_yelp_test,'test')

In [22]:
pd.DataFrame([WordDictionaryCount_Amazon_Train[0],WordDictionaryCount_Amazon_Train[1]])

,bu,uy,ye,er,r,b,be,ew,wa,ar,...,umy,myu,mmc,sez,fja,coz,ozy,kav,gju,ukw
0,1,1,1,11,9,13,5,3,4,7,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,2,5,1,0,0,1,2,...,0,0,0,0,0,0,0,0,0,0


In [23]:
pd.DataFrame([WordDictionaryCount_Yelp_Train[0],WordDictionaryCount_Yelp_Train[1]])

,i,d,do,on,nt,t,k,kn,no,ow,...,nhg,hgn,nwc,sez,ffb,fbe,yec,nww,zeg,ypu
0,4,11,6,13,7,19,1,1,5,1,...,0,0,0,0,0,0,0,0,0,0
1,10,7,6,10,13,21,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


## Menghitung nilai TF pada setiap data Amazon dan Yelp

In [24]:
def computeFrequency(wordDict, ngram):
    freqword = {}
    ngramLength = len(ngram)
    for word, count in wordDict.items():
        freqword[word] = count
    return freqword

In [25]:
def getFrequencyTermPerData(wordDictionary, ngram) :
    CountFrequency = []
    iterator = 0
    while iterator < len(ngram):
        CountFrequency.append(computeFrequency(wordDictionary[iterator], ngram[iterator]))
        iterator = iterator + 1
    
    return CountFrequency

In [26]:
CountFrequency_Sentimen_Amazon_Train = getFrequencyTermPerData(WordDictionaryCount_Amazon_Train, ngram_amazon_train)
CountFrequency_Sentimen_Yelp_Train = getFrequencyTermPerData(WordDictionaryCount_Yelp_Train, ngram_yelp_train)
CountFrequency_Sentimen_Amazon_Test = getFrequencyTermPerData(WordDictionaryCount_Amazon_Test, ngram_amazon_test)
CountFrequency_Sentimen_Yelp_Test = getFrequencyTermPerData(WordDictionaryCount_Yelp_Test, ngram_yelp_test)

## Menghitung nilai IDF pada setiap data Amazon dan Yelp

In [27]:
def computeIDF(WordDict):
    idfDict = {}
    number_of_document_with_term_t_in_it = {}
    N = len(WordDict)
    
    idfDict = dict.fromkeys(WordDict[0].keys(), 0)
    for doc in WordDict:
        for word, val in doc.items():
            if val > 0:
                idfDict[word] += 1
    
    for word, val in idfDict.items():
        number_of_document_with_term_t_in_it[word] = float(val)
        idfDict[word] = np.log((1+N) / (1+val)) + 1
        
    return idfDict, number_of_document_with_term_t_in_it

In [28]:
IDF_Amazon, number_of_document_with_term_t_Amazon_Train = computeIDF(WordDictionaryCount_Amazon_Train)
IDF_Yelp, number_of_document_with_term_t_in_Yelp_Train = computeIDF(WordDictionaryCount_Yelp_Train)

In [29]:
def computeTFIDFnonScale(CountFrequency, idfs):
    tfidf = None
    tfidf = {}
    for word, val in CountFrequency.items():
        tfidf[word] = val*idfs[word]
    return tfidf

In [30]:
def computeTFIDF(tfidf_nonscaled) :
    tfidf_nonscaled = np.array(tfidf_nonscaled)
    tfidf_list = tfidf_nonscaled/sum(tfidf_nonscaled**2)**0.5
    return tfidf_list

In [31]:
# def getTFIDFnonScale(WordDictionary, CountFrequency, IDF):
#     iterator = 0
#     TFIDFnonscale = []
#     while iterator < len(WordDictionary):
#         TFIDFnonscale.append(computeTFIDFnonScale(CountFrequency[iterator], IDF))
#         iterator = iterator + 1
    
#     return TFIDFnonscale

In [32]:
# TFIDF_nonScale_Amazon_Train = getTFIDFnonScale(WordDictionaryCount_Amazon_Train, CountFrequency_Sentimen_Amazon_Train, IDF_Amazon)

In [33]:
# TFIDF_nonScale_Amazon_Test = getTFIDFnonScale(WordDictionaryCount_Amazon_Test, CountFrequency_Sentimen_Amazon_Test, IDF_Amazon)

In [34]:
def getTFIDF(wordDictionary, CountFrequency, IDF):
    TFIDF_non_scaled = None
    TFIDF_non_scaled = []
    iterator = 0
    while iterator < len(wordDictionary):
        TFIDF_non_scaled.append(computeTFIDFnonScale(CountFrequency[iterator], IDF))
        iterator = iterator + 1
    
    TFIDF = []
    iterator = 0
    while iterator < len(wordDictionary):
        tf_idf_list = list(TFIDF_non_scaled[iterator].values())
        TFIDF.append(computeTFIDF(tf_idf_list))
        iterator = iterator + 1
        
    return TFIDF

In [35]:
TFIDF_Amazon_Train = getTFIDF(WordDictionaryCount_Amazon_Train, CountFrequency_Sentimen_Amazon_Train, IDF_Amazon)

In [36]:
TFIDF_Amazon_Test = getTFIDF(WordDictionaryCount_Amazon_Test, CountFrequency_Sentimen_Amazon_Test, IDF_Amazon)

In [37]:
TFIDF_Yelp_Train = getTFIDF(WordDictionaryCount_Yelp_Train, CountFrequency_Sentimen_Yelp_Train, IDF_Yelp)

In [38]:
TFIDF_Yelp_Test = getTFIDF(WordDictionaryCount_Yelp_Test, CountFrequency_Sentimen_Yelp_Test, IDF_Yelp)

In [45]:
def toVectorizeList(TFIDF):
    iterator  = 0
    Vectorized = []
    while iterator < len(TFIDF):
        Vectorized.append(list(TFIDF[iterator]))
        iterator = iterator + 1
    return Vectorized

In [46]:
Vectorizer_Amazon_Train = toVectorizeList(TFIDF_Amazon_Train)
Vectorizer_Amazon_Test = toVectorizeList(TFIDF_Amazon_Test)
Vectorizer_Yelp_Train = toVectorizeList(TFIDF_Yelp_Train)
Vectorizer_Yelp_Test = toVectorizeList(TFIDF_Yelp_Test)

In [47]:
DataFrame_Amazon_Train = pd.DataFrame(Vectorizer_Amazon_Train, columns = Vocabulary_Amazon_Train)
DataFrame_Amazon_Train = DataFrame_Amazon_Train.reindex(sorted(DataFrame_Amazon_Train.columns), axis=1)
print (DataFrame_Amazon_Train)

             a        a    aa        ab        ac        ad   ae        af  \
0     0.174884  0.079896  0.0  0.000000  0.000000  0.000000  0.0  0.000000   
1     0.149341  0.046906  0.0  0.044125  0.052581  0.055971  0.0  0.000000   
2     0.122904  0.056149  0.0  0.026410  0.000000  0.000000  0.0  0.033775   
3     0.107078  0.041393  0.0  0.000000  0.000000  0.000000  0.0  0.000000   
4     0.121365  0.022871  0.0  0.014344  0.017092  0.000000  0.0  0.000000   
...        ...       ...  ...       ...       ...       ...  ...       ...   
7495  0.082949  0.014887  0.0  0.000000  0.000000  0.000000  0.0  0.000000   
7496  0.119551  0.033377  0.0  0.000000  0.000000  0.000000  0.0  0.000000   
7497  0.139474  0.087614  0.0  0.029971  0.000000  0.000000  0.0  0.000000   
7498  0.103292  0.000000  0.0  0.000000  0.072736  0.000000  0.0  0.000000   
7499  0.040737  0.000000  0.0  0.000000  0.000000  0.000000  0.0  0.000000   

            ag   ah  ...  zzf  zzi  zzk  zzl  zzo  zzs  zzt  zz

In [48]:
DataFrame_Amazon_Test = pd.DataFrame(Vectorizer_Amazon_Test, columns = Vocabulary_Amazon_Train)
DataFrame_Amazon_Test = DataFrame_Amazon_Test.reindex(sorted(DataFrame_Amazon_Test.columns), axis=1)
print (DataFrame_Amazon_Test)

             a        a    aa        ab   ac        ad   ae        af  \
0     0.145900  0.061100  0.0  0.019159  0.0  0.048606  0.0  0.024502   
1     0.087064  0.013673  0.0  0.000000  0.0  0.000000  0.0  0.000000   
2     0.090990  0.048992  0.0  0.015362  0.0  0.000000  0.0  0.039293   
3     0.086429  0.000000  0.0  0.000000  0.0  0.000000  0.0  0.000000   
4     0.085100  0.000000  0.0  0.000000  0.0  0.000000  0.0  0.000000   
...        ...       ...  ...       ...  ...       ...  ...       ...   
2495  0.048423  0.015209  0.0  0.000000  0.0  0.000000  0.0  0.000000   
2496  0.051493  0.021564  0.0  0.000000  0.0  0.000000  0.0  0.000000   
2497  0.109906  0.046027  0.0  0.000000  0.0  0.000000  0.0  0.000000   
2498  0.033166  0.000000  0.0  0.000000  0.0  0.000000  0.0  0.000000   
2499  0.089913  0.045185  0.0  0.000000  0.0  0.000000  0.0  0.000000   

            ag   ah  ...  zzf  zzi  zzk  zzl  zzo  zzs  zzt  zzu  zzy  zzz  
0     0.000000  0.0  ...  0.0  0.0  0.0  0.0  

In [49]:
DataFrame_Yelp_Train = pd.DataFrame(Vectorizer_Yelp_Train, columns = Vocabulary_Yelp_Train)
DataFrame_Yelp_Train = DataFrame_Yelp_Train.reindex(sorted(DataFrame_Yelp_Train.columns), axis=1)
print (DataFrame_Yelp_Train)

             a        a    aa        ab       ac        ad   ae        af  \
0     0.171214  0.048025  0.0  0.036015  0.00000  0.017194  0.0  0.014425   
1     0.140388  0.044025  0.0  0.012840  0.00000  0.036779  0.0  0.000000   
2     0.159648  0.054833  0.0  0.011994  0.04579  0.017178  0.0  0.000000   
3     0.190184  0.025403  0.0  0.022225  0.00000  0.000000  0.0  0.026706   
4     0.059365  0.035682  0.0  0.000000  0.00000  0.000000  0.0  0.000000   
...        ...       ...  ...       ...      ...       ...  ...       ...   
7495  0.103979  0.031249  0.0  0.000000  0.00000  0.000000  0.0  0.000000   
7496  0.157036  0.043564  0.0  0.000000  0.00000  0.000000  0.0  0.000000   
7497  0.043994  0.000000  0.0  0.000000  0.00000  0.000000  0.0  0.000000   
7498  0.061627  0.000000  0.0  0.000000  0.00000  0.000000  0.0  0.000000   
7499  0.170437  0.083817  0.0  0.016296  0.00000  0.000000  0.0  0.019582   

       ag   ah  ...  zzb  zzc  zze  zzi  zzl  zzn  zzo  zzu  zzy  zzz  
0  

In [50]:
DataFrame_Yelp_Test = pd.DataFrame(Vectorizer_Yelp_Test, columns = Vocabulary_Yelp_Train)
DataFrame_Yelp_Test = DataFrame_Yelp_Test.reindex(sorted(DataFrame_Yelp_Test.columns), axis=1)
print (DataFrame_Yelp_Test)

             a        a    aa        ab        ac        ad   ae        af  \
0     0.164301  0.026335  0.0  0.023041  0.000000  0.000000  0.0  0.013843   
1     0.078854  0.000000  0.0  0.000000  0.000000  0.000000  0.0  0.000000   
2     0.090357  0.046551  0.0  0.000000  0.000000  0.000000  0.0  0.000000   
3     0.120136  0.009628  0.0  0.000000  0.000000  0.000000  0.0  0.020244   
4     0.164085  0.024055  0.0  0.000000  0.000000  0.000000  0.0  0.010116   
...        ...       ...  ...       ...       ...       ...  ...       ...   
2495  0.120832  0.016139  0.0  0.000000  0.000000  0.000000  0.0  0.000000   
2496  0.090460  0.036248  0.0  0.000000  0.000000  0.090846  0.0  0.000000   
2497  0.181204  0.019803  0.0  0.000000  0.000000  0.000000  0.0  0.041637   
2498  0.169448  0.040739  0.0  0.011881  0.015120  0.000000  0.0  0.014276   
2499  0.154315  0.037101  0.0  0.000000  0.041309  0.000000  0.0  0.000000   

            ag      ah  ...  zzb  zzc  zze  zzi  zzl  zzn  zzo 

## Akurasi dan Waktu Train Test Amazon dan Yelp

In [51]:
Waktu_Training = time()
RF_Classifier_Amazon = RandomForestClassifier(max_depth= 5, n_estimators = 800, random_state=42,
                                       bootstrap = False, min_samples_split = 5, min_samples_leaf = 1, max_features = 'auto')
RF_Classifier_Amazon.fit(Vectorizer_Amazon_Train, Label_Amazon_Train)
print(f"\nWaktu Training Amazon: {round(time()-Waktu_Training, 3)}s")


Waktu Training Amazon: 54.958s


In [52]:
Waktu_Predict_Train = time()
Skor_Train_Amazon = RF_Classifier_Amazon.score(Vectorizer_Amazon_Train, Label_Amazon_Train)
print(f"waktu prediksi (train): {round(time()-Waktu_Predict_Train, 3)}s")

Waktu_Predict_Test = time()
Skor_Test_Amazon = RF_Classifier_Amazon.score(Vectorizer_Amazon_Test, Label_Amazon_Test)
print(f"waktu prediksi (test): {round(time()-Waktu_Predict_Test, 3)}s")

print("\nSkor Random Forest Train Amazon : {}".format(Skor_Train_Amazon))
print("Skor Random Forest Test Amazon : {}".format(Skor_Test_Amazon))

waktu prediksi (train): 6.103s
waktu prediksi (test): 2.092s

Skor Random Forest Train Amazon : 0.8589333333333333
Skor Random Forest Test Amazon : 0.8072


In [53]:
Waktu_Training = time()
RF_Classifier_Yelp = RandomForestClassifier(max_depth= 5, n_estimators = 800, random_state=42,
                                       bootstrap = False, min_samples_split = 5, min_samples_leaf = 1, max_features = 'auto')
RF_Classifier_Yelp.fit(Vectorizer_Yelp_Train, Label_Yelp_Train)
print(f"\nWaktu Training Yelp: {round(time()-Waktu_Training, 3)}s")


Waktu Training Yelp: 59.267s


In [54]:
Waktu_Predict_Train = time()
Skor_Train_Yelp = RF_Classifier_Yelp.score(Vectorizer_Yelp_Train, Label_Yelp_Train)
print(f"waktu prediksi (train): {round(time()-Waktu_Predict_Train, 3)}s")

Waktu_Predict_Test = time()
Skor_Test_Yelp = RF_Classifier_Yelp.score(Vectorizer_Yelp_Test, Label_Yelp_Test)
print(f"waktu prediksi (test): {round(time()-Waktu_Predict_Test, 3)}s")

print("\nSkor Random Forest Train Yelp : {}".format(Skor_Train_Yelp))
print("Skor Random Forest Test Yelp : {}".format(Skor_Test_Yelp))


waktu prediksi (train): 6.649s
waktu prediksi (test): 2.228s

Skor Random Forest Train Yelp : 0.8698666666666667
Skor Random Forest Test Yelp : 0.8424


## Mencari Term-Term yang interseksi antara Amazon dan Yelp

In [55]:
Vocabulary_Intersection = []
iterator = 0
while iterator < len(Vocabulary_Yelp_Train) :
    if Vocabulary_Yelp_Train[iterator] in Vocabulary_Amazon_Train :
        Vocabulary_Intersection.append(Vocabulary_Yelp_Train[iterator])
    iterator = iterator + 1

In [56]:
def RecomputeIDF(WordDictAmazon, WordDictYelp, additional_term_value, Vocabulary_Intersection):
    idfDict = {}
    number_of_document_with_term_t_in_it = {}
    N = len(WordDictYelp) + len(WordDictAmazon)
    idfDict = dict.fromkeys(WordDictYelp[0].keys(), 0)
    for doc in WordDictYelp:
        for word, val in doc.items():
            if val > 0:
                idfDict[word] += 1
    
    for word, val in idfDict.items():
        if word in Vocabulary_Intersection :
            val = val + additional_term_value[word]
        idfDict[word] = np.log((1+N) / (1+val)) + 1
        
    return idfDict

In [57]:
New_IDF_Yelp = RecomputeIDF(WordDictionaryCount_Amazon_Train, WordDictionaryCount_Yelp_Train, number_of_document_with_term_t_Amazon_Train, Vocabulary_Intersection)

In [58]:
New_TFIDF_nonScale_Yelp_Train = []
iterator = 0
while iterator < len(WordDictionaryCount_Yelp_Train):
    New_TFIDF_nonScale_Yelp_Train.append(computeTFIDFnonScale(CountFrequency_Sentimen_Yelp_Train[iterator], New_IDF_Yelp))
    iterator = iterator + 1

In [59]:
New_TFIDF_Yelp_Train = []
iterator = 0
while iterator < len(WordDictionaryCount_Yelp_Train):
    tf_idf_list = list(New_TFIDF_nonScale_Yelp_Train[iterator].values())
    New_TFIDF_Yelp_Train.append(computeTFIDF(tf_idf_list))
    iterator = iterator + 1

In [60]:
New_Vectorizer_Yelp_Train = []
iterator  = 0
while iterator < len(New_TFIDF_Yelp_Train):
    New_Vectorizer_Yelp_Train.append(list(New_TFIDF_Yelp_Train[iterator]))
    iterator = iterator + 1

In [61]:
Waktu_Training = time()
New_RF_Classifier_Yelp = RandomForestClassifier(max_depth= 5, n_estimators = 800, random_state=42,
                                       bootstrap = False, min_samples_split = 5, min_samples_leaf = 1, max_features = 'auto')
New_RF_Classifier_Yelp.fit(New_Vectorizer_Yelp_Train, Label_Yelp_Train)

RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
                       max_depth=5, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, n_estimators=800,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [62]:
Waktu_Predict_Train = time()
New_Skor_Train_Yelp = New_RF_Classifier_Yelp.score(Vectorizer_Yelp_Train, Label_Yelp_Train)
print(f"waktu prediksi (test): {round(time()-Waktu_Predict_Test, 3)}s")
print("Skor Random Forest Test Yelp : {}".format(New_Skor_Train_Yelp))

Waktu_Predict_Test = time()
New_Skor_Test_Yelp = New_RF_Classifier_Yelp.score(Vectorizer_Yelp_Test, Label_Yelp_Test)
print(f"waktu prediksi (test): {round(time()-Waktu_Predict_Test, 3)}s")
print("Skor Random Forest Test Yelp : {}".format(New_Skor_Test_Yelp))

waktu prediksi (test): 176.545s
Skor Random Forest Test Yelp : 0.8629333333333333
waktu prediksi (test): 2.07s
Skor Random Forest Test Yelp : 0.8376
